# Threads

Create a thread

In [8]:
import threading
import time

def do_threading(some_string, some_integer):
    time.sleep(1)
    print("do_threading: some_string =", some_string)
    print("do_threading: some_integer =", some_integer)
    time.sleep(some_integer)
    print("do_threading: exiting...")

my_thread = threading.Thread(target=do_threading, args=('Some arguments', 1))   # use 'daemon=True' to create daemon...
print("Start thread")                                                           # background process which will
my_thread.start()                                                               # terminate when program finishes
print("Returned from start, wait for the thread to complete using join()")
my_thread.join()
print("Returned from join")

Start thread
Returned from start, wait for the thread to complete using join()
do_threading: some_string = Some arguments
do_threading: some_integer = 1
do_threading: exiting...
Returned from join


Using a threaded class

In [9]:
class MyThread(threading.Thread):
    def __init__(self):
        self.main_lock = threading.Lock()
        self.continue_processing = True
        super().__init__()

    def get_continue_processing(self):
        with self.main_lock:
            return self.continue_processing

    def stop_processing(self):
        with self.main_lock:
            self.continue_processing = False

    def run(self):
        while self.get_continue_processing():
            print("MyThread::run: still working")
            time.sleep(0.5)
        print("MyThread::run: stopped working")

my_threaded_class = MyThread()
print("Start the threaded class")
my_threaded_class.start()
print("Go away and do lots of crazy stuff")
time.sleep(3)
print("Ok that's all done, don't need that thread any more")
my_threaded_class.stop_processing()
print("Back from stop_processing")
time.sleep(2)           # let MyThread stop before doing anything else

Start the threaded class
MyThread::run: still working
Go away and do lots of crazy stuff
MyThread::run: still working
MyThread::run: still working
MyThread::run: still working
MyThread::run: still working
MyThread::run: still working
Ok that's all done, don't need that thread any more
Back from stop_processing
MyThread::run: stopped working


Using a thread pool executor for launching multiple threads

In [10]:
import concurrent.futures

def do_some_stuff():
    time.sleep(3)
    print("do_some_stuff: finished what I was doing")
    return 'do_some_stuff return value'

def do_some_other_stuff():
    time.sleep(1)
    print("do_some_other_stuff: finished what I was doing")
    return 'do_some_other_stuff return value'

executor = concurrent.futures.ThreadPoolExecutor(max_workers=2)
print("Submit do_some_stuff")
stuff_future_object = executor.submit(do_some_stuff)
print("Submit do_some_other_stuff")
other_future_object = executor.submit(do_some_other_stuff)
print(stuff_future_object)
print(other_future_object)
print("Wait for threads to complete")
for f in concurrent.futures.as_completed([stuff_future_object, other_future_object]):
    print(f)
    print("Result from this thread =", f.result())

Submit do_some_stuff
Submit do_some_other_stuff
<Future at 0x1b7213774c0 state=running>
<Future at 0x1b720ba58d0 state=running>
Wait for threads to complete
do_some_other_stuff: finished what I was doing
<Future at 0x1b720ba58d0 state=finished returned str>
Result from this thread = do_some_other_stuff return value
do_some_stuff: finished what I was doing
<Future at 0x1b7213774c0 state=finished returned str>
Result from this thread = do_some_stuff return value


Using `map` to launch multiple worker threads

In [11]:
def my_worker_thread(my_string):
    print("my_worker_thread: starting worker thread to calculate length of", my_string)
    if my_string == 'Bob':
        time.sleep(3)
    return f"Length of {my_string} is {len(my_string)}"

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    map_results = executor.map(my_worker_thread, ['Bob', 'Denise', 'Reginald', 'Sidney'])
    for result in map_results:       # will have to wait for 'Bob' to finish as results provided in order given
        print(result)

my_worker_thread: starting worker thread to calculate length of Bob
my_worker_thread: starting worker thread to calculate length of Denise
my_worker_thread: starting worker thread to calculate length of Reginald
my_worker_thread: starting worker thread to calculate length of Sidney
Length of Bob is 3
Length of Denise is 6
Length of Reginald is 8
Length of Sidney is 6


Or as an alternative...

See https://stackoverflow.com/questions/20838162/how-does-threadpoolexecutor-map-differ-from-threadpoolexecutor-submit


In [12]:
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # use list comprehension, no need for map
    mapped_futures = [executor.submit(my_worker_thread, x) for x in ['Bob', 'Denise', 'Reginald', 'Sidney']]
    for f in concurrent.futures.as_completed(mapped_futures):   # get results in order of completion
        print(f.result())

my_worker_thread: starting worker thread to calculate length of Bob
my_worker_thread: starting worker thread to calculate length of Denise
my_worker_thread: starting worker thread to calculate length of Reginald
my_worker_thread: starting worker thread to calculate length of Sidney
Length of Sidney is 6
Length of Reginald is 8
Length of Denise is 6
Length of Bob is 3


Semaphores are usually shared between threads as a way to control resources

In [13]:
my_semaphore = threading.Semaphore(2)       # 2 resources

Starting thread pool with 5 threads and 2 resources

In [19]:
import random

def my_greedy_worker_thread(name, a_semaphore):
    print(f"my_greedy_worker_thread: thread {name} has started!")
    a_semaphore.acquire()
    print(f"my_greedy_worker_thread: thread {name} acquired semaphore for resource!")
    time.sleep(random.randint(1, 3))
    print(f"my_greedy_worker_thread: thread {name} finished with resource so releasing!")
    a_semaphore.release()
    return f'Thread {name} is done'

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # use list comprehension, no need for map
    my_futures = [executor.submit(my_greedy_worker_thread, x, my_semaphore) for x in range(5)]
    for f in concurrent.futures.as_completed(my_futures):   # get results in order of completion
        print(f.result())

def wait_for_an_event(event, number_events):
    events_detected = 0
    while events_detected < number_events:
        print("wait_for_an_event: waiting...")
        event.wait()                    # you can specify a timeout here e.g. wait(timeout=0.5) for 500ms wait
        print("wait_for_an_event: got event")
        event.clear()                   # need to clear events
        events_detected += 1
    print("All events detected, thread terminating...")

my_greedy_worker_thread: thread 0 has started!
my_greedy_worker_thread: thread 0 acquired semaphore for resource!
my_greedy_worker_thread: thread 1 has started!
my_greedy_worker_thread: thread 1 acquired semaphore for resource!
my_greedy_worker_thread: thread 2 has started!
my_greedy_worker_thread: thread 3 has started!
my_greedy_worker_thread: thread 4 has started!
my_greedy_worker_thread: thread 0 finished with resource so releasing!
Thread 0 is done
my_greedy_worker_thread: thread 2 acquired semaphore for resource!
my_greedy_worker_thread: thread 1 finished with resource so releasing!
my_greedy_worker_thread: thread 3 acquired semaphore for resource!
Thread 1 is done
my_greedy_worker_thread: thread 2 finished with resource so releasing!
my_greedy_worker_thread: thread 4 acquired semaphore for resource!
Thread 2 is done
my_greedy_worker_thread: thread 3 finished with resource so releasing!
Thread 3 is done
my_greedy_worker_thread: thread 4 finished with resource so releasing!
Thread 

Using events to signal other threads

In [20]:
my_event = threading.Event()
my_thread = threading.Thread(name='wait_for_an_event', target=wait_for_an_event, args=(my_event, 5))
print("Starting thread")
my_thread.start()
for i in range(5):
    time.sleep(0.5)
    print("Set event")
    my_event.set()
my_thread.join()

Starting thread
wait_for_an_event: waiting...
Set event
wait_for_an_event: got event
wait_for_an_event: waiting...
Set event
wait_for_an_event: got event
wait_for_an_event: waiting...
Set event
wait_for_an_event: got event
wait_for_an_event: waiting...
Set event
wait_for_an_event: got event
wait_for_an_event: waiting...
Set event
wait_for_an_event: got event
All events detected, thread terminating...


Using timers to run a function at some future point in time

In [22]:
def my_function(a_string):
    print(a_string)

timer_thread = threading.Timer(2, my_function,args=("Time is up!",))
timer_thread.start()
print("timer_thread =", str(timer_thread)[1:-1])
timer_thread.join()   # might as well wait in this demo but obviously you might do something interesting here, or not
print("timer_thread =", str(timer_thread)[1:-1])

timer_thread = Timer(Thread-10, started 12528)
Time is up!
timer_thread = Timer(Thread-10, stopped 12528)


Using barriers to ensure threads sync up where necessary

In [23]:
import datetime

def thread_a(a_barrier):
    print("thread_a: started at", datetime.datetime.now())
    time.sleep(2)       # do something that needs to be done for both threads before go any further e.g. client/server
    a_barrier.wait()
    print("thread_a: ok all in sync now, we can start...")
    # do something exciting....
    return f"thread_a: time = {datetime.datetime.now()}"

def thread_b(a_barrier):
    print("thread_b: started at", datetime.datetime.now())
    a_barrier.wait()
    print("thread_b: ok all in sync now, we can start...")
    # do something exciting....
    return f"thread_b: time = {datetime.datetime.now()}"

my_barrier = threading.Barrier(2)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    my_futures = [executor.submit(f, my_barrier) for f in [thread_a, thread_b]]
    for f in concurrent.futures.as_completed(my_futures):
        print(f.result())

thread_a: started at 2022-11-21 23:16:49.696289
thread_b: started at 2022-11-21 23:16:49.705650
thread_a: ok all in sync now, we can start...thread_b: ok all in sync now, we can start...
thread_b: time = 2022-11-21 23:16:51.707899

thread_a: time = 2022-11-21 23:16:51.708872


Define three functions that will run in separate threads

In [24]:
def thread_a(a_condition):
    print("thread_a: started at", datetime.datetime.now())
    with a_condition:
        print("thread_a: creating the shared resource")
        time.sleep(2)           # let's create a shared resource
        print("thread_a: resource now available, notify everyone")
        a_condition.notify_all()    # notify others shared resource is available
        print("thread_a: others now notified")
    # do something exciting....
    return f"thread_a: time = {datetime.datetime.now()}"

def thread_b(a_condition):
    print("thread_b: started at", datetime.datetime.now())
    with a_condition:
        print("thread_b: waiting for shared resource")
        a_condition.wait()
        print("thread_b: we can now use shared resource")
        # do something exciting....
    return f"thread_b: time = {datetime.datetime.now()}"

def thread_c(a_condition):
    print("thread_c: started at", datetime.datetime.now())
    with a_condition:
        print("thread_c: waiting for shared resource")
        a_condition.wait()
        print("thread_c: we can now use shared resource")
        # do something exciting....
    return f"thread_c: time = {datetime.datetime.now()}"

Using conditionals to let other threads know when they can start

In [25]:
my_condition = threading.Condition()
my_notifier_thread = threading.Thread(target=thread_a, args=(my_condition,))
consumer_thread_b = threading.Thread(target=thread_b, args=(my_condition,))
consumer_thread_c = threading.Thread(target=thread_c, args=(my_condition,))

consumer_thread_b.start()
time.sleep(1)
consumer_thread_c.start()
time.sleep(1)
my_notifier_thread.start()

thread_b: started at 2022-11-21 23:18:10.185061
thread_b: waiting for shared resource
thread_c: started at 2022-11-21 23:18:11.324078
thread_c: waiting for shared resource
thread_a: started at 2022-11-21 23:18:12.340645
thread_a: creating the shared resource


thread_a: resource now available, notify everyone
thread_a: others now notified
thread_b: we can now use shared resource
thread_c: we can now use shared resource
